# No regeneration

In [1]:
import numpy as np
import os,sys
import pandas as pd
import time
import random

In [2]:
def cellDivisions(tm,td,ts,stem,positions):
    global n
    for dividingCell in range(0,n):
        tc = ts - tm[dividingCell]
        tr = td[dividingCell] - tc
        if tr < 1/3600:
            tm[dividingCell] = ts
            tm = np.insert(tm,dividingCell,ts)
            tm = tm[:-1]            
            n = n+1
            positions = np.insert(positions,dividingCell,-1)
            td[dividingCell] = np.random.normal(340, 32)
            td = np.insert(td,dividingCell,np.random.normal(340, 32))
            td = td[:-1]               
    return tm,td,stem,positions

In [3]:
def run(steps,tm,td,stem,positions):
    global n
    outgrowth = []
    tracking = []
    tracking.append(positions)
    for ts in range(0,steps+1,1):
        tm,td,stem,positions = cellDivisions(tm,td,ts,stem,positions)
        outgrowth.append(n-n0)    # Appends the outgrowth for each time
        tracking.append(positions)
    return outgrowth,tracking

In [4]:
root = '/home/ecuracosta/Desktop/MS/Simulations/No_regeneration/'
os.chdir(root)

start_time = time.time() 

emptySpace = 250
steps = 24*8    # Number of steaps (in hours)
n0 = 205    # Number of cells after amputation

lattice = n0+emptySpace   
folder = 'n0='+str(n0)
os.makedirs(folder)
print('n0:',n0)
for seed in range(0,200):
    n = n0
    positions = np.arange(0,n0) 
    tm = np.random.uniform(-(340-2*32),0,lattice)
    td = np.random.normal(340,32,lattice)
    stem = np.zeros(lattice)
    solution,track = run(steps,tm,td,stem,positions)

    d = {"time":np.arange(0,steps+1),"outgrowth":solution}
    df = pd.DataFrame(d)
    outfile = open(str(root)+str(folder)+'/'+'sim_seed='+str(seed)+'.csv', 'a')
    df.to_csv(outfile, sep=',')
    outfile.close()

print("Total time: %s seconds <--" % (time.time() - start_time))

n0: 205
Total time: 8.649936437606812 seconds <--
